# DataLab Quickstart
This notebook demonstrates how to connect to the shared `interactions.db` file,
normalize the typing metadata JSON, and compute lightweight aggregates.

In [4]:
from __future__ import annotations

import json
from pathlib import Path
import sqlite3

import pandas as pd

ROOT = Path.cwd().parents[1]
DB_PATH = str(ROOT / 'data' / 'interactions.db')
DB_PATH

'D:\\Files\\Code 3\\ChatAI-DataLab\\data\\interactions.db'

In [5]:
conn = sqlite3.connect(DB_PATH)
df = pd.read_sql_query("SELECT * FROM interactions ORDER BY created_at DESC", conn)
df.head()

,id,user_prompt_text,typing_metadata_json,ai_response_text,model_name,latency_ms,created_at
0,d86a4287-a355-4dce-ba65-a119deb028ed,hffgfhgfghfvvvhkgghgfftff,"{""total_duration_ms"": 17841, ""token_estimate"":...",[echo] hffgfhgfghfvvvhkgghgfftff,echo,0,2025-11-15 13:04:03.816895
1,30dd2b92-e93a-49f0-a50a-68768f827ffb,hi there. how are you?,"{""total_duration_ms"": 22225, ""token_estimate"":...",[echo] hi there. how are you?,echo,58,2025-11-15 13:02:41.070533
2,2a5ddc08-b392-402a-a3f7-a53f93926935,cgcc,"{""total_duration_ms"": 3198, ""token_estimate"": ...",[echo] cgcc,echo,27,2025-11-15 11:04:14.875909
3,33f1a4ea-6063-4757-95c5-e315632afe1f,ccccczaszcxsc xdxscxc,"{""total_duration_ms"": 4994, ""token_estimate"": ...",[echo] ccccczaszcxsc xdxscxc,echo,0,2025-11-15 09:38:16.355565
4,e3002cfc-ed04-4818-b4dc-787fa439e3e9,ijhhhipkkkp,"{""total_duration_ms"": 3362, ""token_estimate"": ...",[echo] ijhhhipkkkp,echo,1,2025-11-15 09:30:56.005823


In [6]:
def explode_pauses(row):
    events = json.loads(row['typing_metadata_json']).get('pause_events', [])
    total_pause = sum(item['duration_ms'] for item in events)
    return pd.Series({
        'pause_count': len(events),
        'total_pause_ms': total_pause
    })

features = df.apply(explode_pauses, axis=1) if not df.empty else pd.DataFrame(columns=['pause_count', 'total_pause_ms'])
features.head()

,pause_count,total_pause_ms
0,4,11149
1,8,12924
2,1,1997
3,2,703
4,2,1119
